# INFO 2950 Final Project

In [2]:
import pandas as pd
import seaborn

In [3]:
%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

%sql duckdb:///:memory:

reserach question:

In [6]:
pd.read_csv('./Data_2019/Incidents_Offenses_Victims_and_Known_Offenders_by_Bias_Motivation_2019.csv')

,Bias motivation,Incidents,Offenses,Victims1,Known\noffenders2
0,Total,"7,314","8,559","8,812","6,406"
1,Single-Bias Incidents,"7,103","8,302","8,552","6,268"
2,Race/Ethnicity/Ancestry:,"3,963","4,784","4,930","3,550"
3,Anti-White,666,755,775,645
4,Anti-Black or African American,"1,930","2,314","2,391","1,682"
5,Anti-American Indian or Alaska Native,119,126,135,101
6,Anti-Asian,158,205,215,153
7,Anti-Native Hawaiian or Other Pacific Islander,21,25,26,26
8,"Anti-Multiple Races, Group",134,171,173,82
9,Anti-Arab,95,122,126,86


In [5]:
pd.read_csv('./Data_2020/Incidents_Offenses_Victims_and_Known_Offenders_by_Bias_Motivation_2020.csv')

,Bias motivation,Incidents,Offenses,Victims1,Known Offenders2
0,Total,"8,263","11,129","11,472","6,780"
1,Single-Bias Incidents,"8,052","10,790","11,126","6,657"
2,Race/Ethnicity/Ancestry,"5,227","6,677","6,880","4,339"
3,Anti-White,869,"1,048","1,082",825
4,Anti-Black or African American,"2,871","3,819","3,915","2,302"
5,Anti-American Indian or Alaska Native,96,103,108,74
6,Anti-Asian,279,330,342,239
7,Anti-Native Hawaiian or Other Pacific Islander,15,15,18,12
8,"Anti-Multiple Races, Group",211,270,281,114
9,Anti-Arab,71,85,87,52
